# Clustering
## Ikjot Singh
### 102116071
### 3CS11

### Imports
*Importing libraries*

In [1]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
from pycaret.clustering import *

*Importing Dataset*

Estimation of obesity levels based on eating habits and physical condition . (2019). UCI Machine Learning Repository. https://doi.org/10.24432/C5H31Z.

In [2]:
estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition = fetch_ucirepo(id=544) 

### Preprocessing

*Splitting Dataset into x and y* 

In [3]:
X = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.features 
y = estimation_of_obesity_levels_based_on_eating_habits_and_physical_condition.data.targets

In [4]:
X.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation


In [5]:
y.head()


,NObeyesdad
0,Normal_Weight
1,Normal_Weight
2,Normal_Weight
3,Overweight_Level_I
4,Overweight_Level_II


*Preprocessing Data*

In [6]:
X["Gender"]=X["Gender"].apply(lambda x: {True: 1, False:0}[x=="Male"])
X["FAVC"]=X["FAVC"].apply(lambda x: {True: 1, False:0}[x=="yes"])
X["family_history_with_overweight"]=X["family_history_with_overweight"].apply(lambda x: {True: 1, False:0}[x=="yes"])
X["SMOKE"]=X["SMOKE"].apply(lambda x: {True: 1, False:0}[x=="yes"])
X["SCC"]=X["SCC"].apply(lambda x: {True: 1, False:0}[x=="yes"])

In [7]:
X['CAEC']=X['CAEC'].apply(lambda x: {'no':0,'Sometimes':1,'Frequently':2,'Always':3}[x])
X['CALC']=X['CALC'].apply(lambda x: {'no':0,'Sometimes':1,'Frequently':2,'Always':3}[x])
X=pd.get_dummies(X,columns=['MTRANS'],drop_first=True)

### Metrics

*Creating dataframes for storing metrics*

In [8]:
rows = [ 'Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin']
kmeans_metrics = pd.DataFrame(index = rows)
hier_metrics = pd.DataFrame(index = rows)
kmeans_shift_metrics = pd.DataFrame(index = rows)

*Defining a function to get the metrics*

In [9]:
def get_scores(Model, metrics):
    silhouette_score_function = metrics.loc['silhouette', 'Score Function']
    silhouette_score = silhouette_score_function(X, Model.labels_)

    chs_score_function = metrics.loc['chs', 'Score Function']
    Calinski_Harabasz_score = chs_score_function(X, Model.labels_)

    db_score_function = metrics.loc['db', 'Score Function']
    Davies_Bouldin_score = db_score_function(X, Model.labels_)

    return (silhouette_score, Calinski_Harabasz_score, Davies_Bouldin_score)

### Running Models

*Defining a function to run models*

In [10]:
def run_clustering_model(model_name, setup_args, num_clusters_range, data):
    metrics_dict = {}

    for setup_arg in setup_args:
        for num_clusters in num_clusters_range:
            print(model_name, setup_arg['Type'], num_clusters)
            model_params = setup(data=X, **setup_arg['setup_kwargs'])
            model = create_model(model_name, num_clusters=num_clusters)
            metrics = get_metrics()

            a,b,c = get_scores(model, metrics)

            metrics_dict[(setup_arg['Type'], f'c={num_clusters}')] = [a,b,c]

    return pd.DataFrame(data=metrics_dict, index=rows)


*Defining rows for metrics and number of clusters*

In [11]:
rows = ['Silhouette', 'Calinski-Harabasz', 'Davies-Bouldin']
num_clusters_range = range(3, 6) 

*Defining setup args for all scenarios*

In [12]:

# List of dictionaries containing the different arguments for setup function
setup_args = [
    {'Type': 'No Data Preprocessing', 'setup_kwargs': {'verbose': False}},
    {'Type': 'Using Normalization', 'setup_kwargs': {'normalize': True, 'normalize_method': 'zscore', 'verbose': False}},
    {'Type': 'Using Transform', 'setup_kwargs': {'transformation': True, 'transformation_method': 'yeo-johnson', 'verbose': False}},
    {'Type': 'Using PCA', 'setup_kwargs': {'pca': True, 'pca_method': 'linear', 'verbose': False}},
    {'Type': 'Using T+N', 'setup_kwargs': {'transformation': True, 'transformation_method': 'yeo-johnson', 'normalize': True, 'normalize_method': 'zscore', 'verbose': False}},
    {'Type': 'Using T+N+PCA', 'setup_kwargs': {'pca': True, 'pca_method': 'linear', 'normalize': True, 'normalize_method': 'zscore', 'transformation': True, 'transformation_method': 'yeo-johnson', 'verbose': False}}
]



*Creating a dictionary to store results*

In [13]:
result_dict={}

*Running Models*

In [14]:
models = ['kmeans', 'hclust', 'meanshift']
for model in models:
    result_dict[model] = run_clustering_model(model_name=model, setup_args=setup_args, num_clusters_range=num_clusters_range, data=X)

kmeans No Data Preprocessing 3


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.5031,4729.2488,0.6675,0,0,0


kmeans No Data Preprocessing 4


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.4751,4818.2893,0.7003,0,0,0


kmeans No Data Preprocessing 5


...

,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3182,36.0077,0.7927,0,0,0


meanshift Using T+N+PCA 4


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3182,36.0077,0.7927,0,0,0


meanshift Using T+N+PCA 5


,Silhouette,Calinski-Harabasz,Davies-Bouldin,Homogeneity,Rand Index,Completeness
0,0.3182,36.0077,0.7927,0,0,0


### Results
*Kmeans Clustering*

In [15]:
result_dict['kmeans']

No Data Preprocessing                            \
                                    c=3          c=4          c=5   
Silhouette                     0.503127     0.475092     0.431226   
Calinski-Harabasz           4729.250305  4818.286402  4599.036346   
Davies-Bouldin                 0.667549     0.700300     0.756829   

                  Using Normalization                         Using Transform  \
                                  c=3         c=4         c=5             c=3   
Silhouette                   0.155356   -0.039229   -0.007025        0.062337   
Calinski-Harabasz          838.228756  444.686679  446.775732      384.602450   
Davies-Bouldin               1.854982    4.365204    2.319714        2.814921   

                                             Using PCA               \
                          c=4         c=5          c=3          c=4   
Silhouette          -0.098254   -0.128453     0.503127     0.475092   
Calinski-Harabasz  277.110536  216.554677  4729.250305  4818.286402   
Davies-Bouldin       3.757068    8.267418     0.667549     0.700300   

                                 Using T+N                          \
                           c=5         c=3         c=4         c=5   
Silhouette            0.431226    0.146546   -0.026773   -0.005986   
Calinski-Harabasz  4599.036346  808.964756  438.170403  439.472583   
Davies-Bouldin        0.756829    1.830030    4.735321    2.458323   

                  Using T+N+PCA                          
                            c=3         c=4         c=5  
Silhouette             0.148261   -0.040432   -0.050791  
Calinski-Harabasz    818.502893  433.653896  312.546238  
Davies-Bouldin         1.816003    4.575594    6.332252

*Hierarchial Clustering*

In [16]:
result_dict['hclust']

No Data Preprocessing                            \
                                    c=3          c=4          c=5   
Silhouette                     0.475726     0.467548     0.415117   
Calinski-Harabasz           4302.495839  4346.043925  4114.637298   
Davies-Bouldin                 0.700061     0.666997     0.731406   

                  Using Normalization                         Using Transform  \
                                  c=3         c=4         c=5             c=3   
Silhouette                   0.060391   -0.029765   -0.042453        0.062337   
Calinski-Harabasz          436.363741  309.647539  232.338639      384.602450   
Davies-Bouldin               2.471153    2.976022    3.615776        2.814921   

                                             Using PCA               \
                          c=4         c=5          c=3          c=4   
Silhouette          -0.099628   -0.129376     0.475726     0.467548   
Calinski-Harabasz  276.709678  216.276108  4302.495839  4346.043925   
Davies-Bouldin       3.737062    8.253889     0.700061     0.666997   

                                 Using T+N                          \
                           c=5         c=3         c=4         c=5   
Silhouette            0.415117    0.098841   -0.014811   -0.029596   
Calinski-Harabasz  4114.637298  392.483965  266.722296  200.142280   
Davies-Bouldin        0.731406    4.727314    4.503711    4.842121   

                  Using T+N+PCA                          
                            c=3         c=4         c=5  
Silhouette             0.098841   -0.014811   -0.029596  
Calinski-Harabasz    392.483965  266.722296  200.142280  
Davies-Bouldin         4.727314    4.503711    4.842121

*Kmeans Shift*

In [17]:
result_dict['meanshift']

No Data Preprocessing                            \
                                    c=3          c=4          c=5   
Silhouette                     0.559179     0.559179     0.559179   
Calinski-Harabasz           4309.498187  4309.498187  4309.498187   
Davies-Bouldin                 0.598328     0.598328     0.598328   

                  Using Normalization                     Using Transform  \
                                  c=3       c=4       c=5             c=3   
Silhouette                  -0.740924 -0.740924 -0.740924        0.019492   
Calinski-Harabasz            4.867212  4.867212  4.867212      158.963052   
Davies-Bouldin               2.757867  2.757867  2.757867        1.720077   

                                             Using PCA               \
                          c=4         c=5          c=3          c=4   
Silhouette           0.019492    0.019492     0.559179     0.559179   
Calinski-Harabasz  158.963052  158.963052  4309.498187  4309.498187   
Davies-Bouldin       1.720077    1.720077     0.598328     0.598328   

                               Using T+N                     Using T+N+PCA  \
                           c=5       c=3       c=4       c=5           c=3   
Silhouette            0.559179 -0.742555 -0.742555 -0.742555     -0.742555   
Calinski-Harabasz  4309.498187  5.356018  5.356018  5.356018      5.356018   
Davies-Bouldin        0.598328  3.614551  3.614551  3.614551      3.614551   

                                       
                        c=4       c=5  
Silhouette        -0.742555 -0.742555  
Calinski-Harabasz  5.356018  5.356018  
Davies-Bouldin     3.614551  3.614551